# Join Race Results
### tables to join - circuits, drivers, constructors, races, results

In [0]:
%run "../includes/configuration"

In [0]:
display(dbutils.fs.ls(processed_folder_path))

In [0]:
races_df = spark.read.parquet(f'{processed_folder_path}/races')

In [0]:
display(races_df)

In [0]:
circuits_df = spark.read.parquet(f'{processed_folder_path}/circuits')

In [0]:
display(circuits_df)

In [0]:
races_circuits_df = races_df.join(circuits_df, races_df.circuit_id == circuits_df.circuit_id, "inner")\
    .select(
        races_df.race_id,
        races_df.race_year,
        races_df.name.alias('race_name'),
        races_df.race_timestamp.alias('race_date'),
        circuits_df.location.alias('circuit_location')
        )

In [0]:
display(races_circuits_df)

In [0]:
drivers_df = spark.read.parquet(f'{processed_folder_path}/drivers')

In [0]:
display(drivers_df)

In [0]:
from pyspark.sql.functions import col

In [0]:
results_df = spark.read.parquet(f'{processed_folder_path}/results')\
    .select(
        "driver_id",
        "race_id",
        "constructor_id",
        "grid",
        col("fastest_lap_time").alias("fastest_lap"),
        col("time").alias("race_time"),
        "points"
    )

In [0]:
results_df.printSchema()

In [0]:
display(results_df)

In [0]:
drivers_df = spark.read.parquet(f'{processed_folder_path}/drivers')\
    .select(
        "driver_id",
        col("name").alias("driver_name"),
        col("number").alias("driver_number"),
        col("nationality").alias("driver_nationality")
    )

In [0]:
drivers_df.printSchema()

In [0]:
drivers_results_df = results_df.join(drivers_df, results_df.driver_id == drivers_df.driver_id, "inner").drop(results_df.driver_id, drivers_df.driver_id)

In [0]:
display(drivers_results_df)

In [0]:
constructors_df = spark.read.parquet(f'{processed_folder_path}/constructors')\
    .select(
        "constructor_id",
        col("name").alias("team")
    )

In [0]:
constructors_df.printSchema()

In [0]:
drivers_results_constructors_df = drivers_results_df.join(constructors_df, drivers_results_df.constructor_id ==constructors_df.constructor_id, "inner").drop( drivers_results_df.constructor_id, constructors_df.constructor_id)

In [0]:
display(drivers_results_constructors_df)

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
final_df = drivers_results_constructors_df.join(races_circuits_df, drivers_results_constructors_df.race_id == races_circuits_df.race_id, "inner").drop(drivers_results_constructors_df.race_id ,races_circuits_df.race_id).withColumn("created_date", current_timestamp())

In [0]:
final_df.printSchema()

In [0]:
display(final_df.filter("race_year == 2020 and race_name == 'Abu Dhabi Grand Prix'").orderBy("points", ascending=False))